In [108]:
import pandas as pd
from pandas.io import gbq
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import numpy as np
import pandas_gbq
from sklearn.impute import KNNImputer

In [129]:
fut_index = ['Nifty','BankNifty']

In [131]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

final_filled_df = pd.DataFrame()

for i in fut_index:
    print(i)
    sql = "SELECT * from `reddy000-c898c.Titania_Dataset.futures_options_signals` where Stock = '"+str(i)+"' order by Datetime asc"
    
    df = pandas_gbq.read_gbq(sql, project_id='reddy000-c898c')
    
    df = pd.DataFrame(df)
    
    df = df[['Datetime', 'Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio','Call_Majority', 'Put_Majority','Stock']]
    
    datetime_values = df['Datetime']
    
    print(df.columns)
    
    print(df.head(5))
    
    print(df.isnull().sum()/len(df))
    
    print(df.dtypes)

    df["call_changeinopeninterest"] = df["call_changeinopeninterest"].replace(0, np.nan)
    df["put_changeinopeninterest"] = df["put_changeinopeninterest"].replace(0, np.nan)

    df["call_traded_volume"] = df["call_traded_volume"].replace(0, np.nan)
    df["put_traded_volume"] = df["put_traded_volume"].replace(0, np.nan)

    df["call_pchange"] = df["call_pchange"].replace(0, np.nan)
    df["put_pchange"] = df["put_pchange"].replace(0, np.nan)
    
    df["call_changeinopeninterest"] = df["call_changeinopeninterest"].replace(0, np.nan)
    df["put_changeinopeninterest"] = df["put_changeinopeninterest"].replace(0, np.nan)

    df["call_traded_volume"] = df["call_traded_volume"].replace(0, np.nan)
    df["put_traded_volume"] = df["put_traded_volume"].replace(0, np.nan)

    df["call_pchange"] = df["call_pchange"].replace(0, np.nan)
    df["put_pchange"] = df["put_pchange"].replace(0, np.nan)
    
    df['Stock'] = df.Stock.map({'Nifty':1,'BankNifty':0})
    df['Call_Majority'] = df.Call_Majority.map({'Buyers':0,'Sellers':1})
    df['Put_Majority'] = df.Put_Majority.map({'Buyers':0,'Sellers':1})


    imputer = KNNImputer(n_neighbors = 3)

    df_filled = imputer.fit_transform(df[['Strike_Price', 'future_volume', 'call_traded_volume',
           'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
           'put_pchange', 'put_changeinopeninterest', 'pcr_ratio','Stock','Call_Majority', 'Put_Majority']])
    
    df_filled = pd.DataFrame(df_filled,columns = ['Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio','Stock','Call_Majority', 'Put_Majority'])
    
    df_filled['Datetime'] = datetime_values
    
    df_filled['Stock'] = df.Stock.map({1:'Nifty',0:'BankNifty'})

    df_filled["Call_Majority"] = np.where(df_filled["Call_Majority"] > 0.5, "Sellers", "Buyers")
    df_filled["Put_Majority"] = np.where(df_filled["Put_Majority"] > 0.5, "Sellers", "Buyers")
    
    print(df_filled.isnull().sum()/len(df_filled))
    
    
    df_filled['call_value'] = ['Long Buildups' if x>0 and y>0 else 'Long Unwinding' if x<0 and y <0 else 'Shorts Buildups' if x < 0 and y > 0 else 'Shorts Coverings' if x > 0 and y < 0 else '-' for x,y in zip(df_filled['call_pchange'],df_filled['call_changeinopeninterest'])]
    df_filled['put_value'] = ['Long Buildups' if x>0 and y>0 else 'Long Unwinding' if x<0 and y <0 else 'Shorts Buildups' if x < 0 and y > 0 else 'Shorts Coverings' if x > 0 and y < 0 else '-' for x,y in zip(df_filled['put_pchange'],df_filled['put_changeinopeninterest'])]
    
    
    final_filled_df = final_filled_df.append(df_filled)
    
    

Nifty


Downloading: 100%|██████████████████████| 1875/1875 [00:00<00:00, 3434.32rows/s]


Index(['Datetime', 'Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio', 'Call_Majority',
       'Put_Majority', 'Stock'],
      dtype='object')
                     Datetime Strike_Price  future_volume  call_traded_volume  \
0  2022-11-01 09:15:00.000000        18150         251850                 0.0   
1  2022-11-01 09:16:00.000000        18150         130500                 0.0   
2  2022-11-01 09:17:00.000000        18150         129750              7495.0   
3  2022-11-01 09:18:00.000000        18150          91300             30163.0   
4  2022-11-01 09:19:00.000000        18150          58300             41953.0   

   call_pchange  call_changeinopeninterest  put_traded_volume  put_pchange  \
0    229.749104                        0.0                0.0          -50   
1    229.749104                        0.0                0.0          -50 

/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_40377/2141261545.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_filled_df = final_filled_df.append(df_filled)


BankNifty


Downloading: 100%|██████████████████████| 1875/1875 [00:00<00:00, 2900.07rows/s]


Index(['Datetime', 'Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio', 'Call_Majority',
       'Put_Majority', 'Stock'],
      dtype='object')
                     Datetime Strike_Price  future_volume  call_traded_volume  \
0  2022-11-01 09:15:00.000000        41600          68625            872790.0   
1  2022-11-01 09:16:00.000000        41600          43200                 NaN   
2  2022-11-01 09:17:00.000000        41600          45750             20091.0   
3  2022-11-01 09:18:00.000000        41600          22625             45638.0   
4  2022-11-01 09:19:00.000000        41600          15800             45638.0   

   call_pchange  call_changeinopeninterest  put_traded_volume  put_pchange  \
0     10.935024                    -3344.0                0.0          -36   
1      0.000000                        0.0                NaN            0 

/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_40377/2141261545.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_filled_df = final_filled_df.append(df_filled)


In [133]:
final_filled_df.reset_index(inplace=True,drop=True)

In [135]:
destination_table = 'Titania_Dataset.' + str('knn_imputed_futures_options_signals')


final_filled_df.to_gbq(destination_table=destination_table,
            project_id='reddy000-c898c',
           if_exists='replace')

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2680.07it/s]


In [109]:
import pandas_gbq

sql = """
SELECT * FROM `reddy000-c898c.Titania_Dataset.futures_options_signals` 
where stock = 'Nifty'
order by Datetime asc
"""
df = pandas_gbq.read_gbq(sql, project_id='reddy000-c898c')



Downloading: 100%|██████████████████████| 1875/1875 [00:00<00:00, 3128.88rows/s]


In [110]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.DataFrame(df)

df = df[['Datetime', 'Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio','Call_Majority', 'Put_Majority','Stock']]

In [111]:
datetime_values = df['Datetime']

In [112]:
print(df.columns)
print(df.head(5))

Index(['Datetime', 'Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio', 'Call_Majority',
       'Put_Majority', 'Stock'],
      dtype='object')
                     Datetime Strike_Price  future_volume  call_traded_volume  \
0  2022-11-01 09:15:00.000000        18150         251850                 0.0   
1  2022-11-01 09:16:00.000000        18150         130500                 0.0   
2  2022-11-01 09:17:00.000000        18150         129750              7495.0   
3  2022-11-01 09:18:00.000000        18150          91300             30163.0   
4  2022-11-01 09:19:00.000000        18150          58300             41953.0   

   call_pchange  call_changeinopeninterest  put_traded_volume  put_pchange  \
0    229.749104                        0.0                0.0          -50   
1    229.749104                        0.0                0.0          -50 

In [113]:
df.describe()

,future_volume,call_traded_volume,call_pchange,call_changeinopeninterest,put_traded_volume,put_pchange,put_changeinopeninterest,pcr_ratio
count,1875.000000,1.702000e+03,1875.000000,1875.000000,1.702000e+03,1875.000000,1875.000000,1700.000000
mean,19283.177600,1.877845e+06,-5.588909,69314.248000,1.018112e+06,-12.968533,22161.921067,0.626529
std,20548.745273,2.419400e+06,66.848687,50055.606305,1.074901e+06,23.699438,28704.584082,0.260443
min,1508.000000,0.000000e+00,-99.938272,0.000000,0.000000e+00,-58.000000,-80601.000000,0.100000
25%,7600.000000,7.575640e+05,-37.339602,40371.000000,3.848225e+05,-32.000000,4394.500000,0.400000
50%,13350.000000,1.127954e+06,-19.783352,53601.000000,7.301120e+05,-13.000000,27159.000000,0.700000
75%,23275.000000,1.581183e+06,0.000000,95738.000000,1.063087e+06,0.000000,45946.000000,0.820000
max,289150.000000,1.296274e+07,229.749104,228445.000000,5.182542e+06,47.000000,134889.000000,2.240000


In [114]:
df.isnull().sum()/len(df)

Datetime                     0.000000
Strike_Price                 0.000000
future_volume                0.000000
call_traded_volume           0.092267
call_pchange                 0.000000
call_changeinopeninterest    0.000000
put_traded_volume            0.092267
put_pchange                  0.000000
put_changeinopeninterest     0.000000
pcr_ratio                    0.093333
Call_Majority                0.092267
Put_Majority                 0.092267
Stock                        0.000000
dtype: float64

In [115]:
print(df.dtypes)

df["call_changeinopeninterest"] = df["call_changeinopeninterest"].replace(0, np.nan)
df["put_changeinopeninterest"] = df["put_changeinopeninterest"].replace(0, np.nan)

df["call_traded_volume"] = df["call_traded_volume"].replace(0, np.nan)
df["put_traded_volume"] = df["put_traded_volume"].replace(0, np.nan)

df["call_pchange"] = df["call_pchange"].replace(0, np.nan)
df["put_pchange"] = df["put_pchange"].replace(0, np.nan)



Datetime                      object
Strike_Price                  object
future_volume                  Int64
call_traded_volume           float64
call_pchange                 float64
call_changeinopeninterest    float64
put_traded_volume            float64
put_pchange                    Int64
put_changeinopeninterest       Int64
pcr_ratio                    float64
Call_Majority                 object
Put_Majority                  object
Stock                         object
dtype: object


In [116]:
df['call_traded_volume'].isnull().sum()

175

In [117]:
df.tail(5)

,Datetime,Strike_Price,future_volume,call_traded_volume,call_pchange,call_changeinopeninterest,put_traded_volume,put_pchange,put_changeinopeninterest,pcr_ratio,Call_Majority,Put_Majority,Stock
1870,2022-11-07 15:25:00.000000,18250,57900,1166693.0,-0.447094,39917.0,459665.0,-44,7367,0.39,Sellers,Sellers,Nifty
1871,2022-11-07 15:26:00.000000,18250,40800,1179895.0,-2.384501,39917.0,464344.0,-43,7367,0.39,Sellers,Sellers,Nifty
1872,2022-11-07 15:27:00.000000,18250,53750,1179895.0,-2.384501,39917.0,464344.0,-43,7367,0.39,Sellers,Sellers,Nifty
1873,2022-11-07 15:28:00.000000,18250,47200,1186745.0,-1.862891,36795.0,467703.0,-43,8185,0.39,Sellers,Sellers,Nifty
1874,2022-11-07 15:29:00.000000,18250,1508,1192522.0,-2.980626,30809.0,470518.0,-42,7174,0.39,Sellers,Sellers,Nifty


In [118]:
from sklearn.impute import KNNImputer

df['Stock'] = df.Stock.map({'Nifty':1,'BankNifty':0})
df['Call_Majority'] = df.Call_Majority.map({'Buyers':0,'Sellers':1})
df['Put_Majority'] = df.Put_Majority.map({'Buyers':0,'Sellers':1})


imputer = KNNImputer(n_neighbors = 3)

df_filled = imputer.fit_transform(df[['Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio','Stock','Call_Majority', 'Put_Majority']])

In [119]:
df_filled = pd.DataFrame(df_filled,columns = ['Strike_Price', 'future_volume', 'call_traded_volume',
       'call_pchange', 'call_changeinopeninterest', 'put_traded_volume',
       'put_pchange', 'put_changeinopeninterest', 'pcr_ratio','Stock','Call_Majority', 'Put_Majority'])

In [120]:
df_filled['Datetime'] = datetime_values

In [122]:
df_filled['Stock'] = df.Stock.map({1:'Nifty',0:'BankNifty'})

df_filled["Call_Majority"] = np.where(df_filled["Call_Majority"] > 0.5, "Sellers", "Buyers")
df_filled["Put_Majority"] = np.where(df_filled["Put_Majority"] > 0.5, "Sellers", "Buyers")

In [124]:
df_filled.isnull().sum()/len(df_filled)

Strike_Price                 0.0
future_volume                0.0
call_traded_volume           0.0
call_pchange                 0.0
call_changeinopeninterest    0.0
put_traded_volume            0.0
put_pchange                  0.0
put_changeinopeninterest     0.0
pcr_ratio                    0.0
Stock                        0.0
Call_Majority                0.0
Put_Majority                 0.0
Datetime                     0.0
dtype: float64

In [125]:
df_filled['call_value'] = ['Long Buildups' if x>0 and y>0 else 'Long Unwinding' if x<0 and y <0 else 'Shorts Buildups' if x < 0 and y > 0 else 'Shorts Coverings' if x > 0 and y < 0 else '-' for x,y in zip(df_filled['call_pchange'],df_filled['call_changeinopeninterest'])]
df_filled['put_value'] = ['Long Buildups' if x>0 and y>0 else 'Long Unwinding' if x<0 and y <0 else 'Shorts Buildups' if x < 0 and y > 0 else 'Shorts Coverings' if x > 0 and y < 0 else '-' for x,y in zip(df_filled['put_pchange'],df_filled['put_changeinopeninterest'])]

In [126]:
df_filled

,Strike_Price,future_volume,call_traded_volume,call_pchange,call_changeinopeninterest,put_traded_volume,put_pchange,put_changeinopeninterest,pcr_ratio,Stock,Call_Majority,Put_Majority,Datetime,call_value,put_value
0,18150.0,251850.0,3.522053e+05,229.749104,22723.666667,5.783453e+05,-50.000000,10560.333333,1.103333,Nifty,Buyers,Sellers,2022-11-01 09:15:00.000000,Long Buildups,Shorts Buildups
1,18150.0,130500.0,2.833000e+05,229.749104,33014.666667,1.547490e+05,-50.000000,20095.333333,0.490000,Nifty,Buyers,Sellers,2022-11-01 09:16:00.000000,Long Buildups,Shorts Buildups
2,18150.0,129750.0,7.495000e+03,48.708920,7485.000000,2.675000e+03,-37.000000,4143.666667,0.360000,Nifty,Buyers,Buyers,2022-11-01 09:17:00.000000,Long Buildups,Shorts Buildups
3,18150.0,91300.0,3.016300e+04,56.103286,7485.000000,1.124700e+04,-38.000000,4143.666667,0.370000,Nifty,Buyers,Buyers,2022-11-01 09:18:00.000000,Long Buildups,Shorts Buildups
4,18150.0,58300.0,4.195300e+04,55.399061,7485.000000,1.693400e+04,-39.000000,4143.666667,0.400000,Nifty,Buyers,Buyers,2022-11-01 09:19:00.000000,Long Buildups,Shorts Buildups
5,18150.0,76700.0,4.195300e+04,55.399061,7485.000000,1.693400e+04,-39.000000,4143.666667,0.400000,Nifty,Buyers,Buyers,2022-11-01 09:20:00.000000,Long Buildups,Shorts Buildups
6,18150.0,50000.0,4.824800e+04,55.516432,6078.000000,2.012800e+04,-39.000000,3188.000000,0.420000,Nifty,Buyers,Buyers,2022-11-01 09:21:00.000000,Long Buildups,Shorts Buildups
7,18150.0,35150.0,5.440000e+04,54.225352,6078.000000,2.275000e+04,-40.000000,3188.000000,0.420000,Nifty,Buyers,Buyers,2022-11-01 09:22:00.000000,Long Buildups,Shorts Buildups
8,18150.0,54300.0,6.527000e+04,51.291080,10299.000000,2.914000e+04,-39.000000,6055.000000,0.450000,Nifty,Buyers,Buyers,2022-11-01 09:23:00.000000,Long Buildups,Shorts Buildups
9,18150.0,41150.0,7.247300e+04,56.572770,10299.000000,3.210400e+04,-40.000000,6055.000000,0.440000,Nifty,Buyers,Buyers,2022-11-01 09:24:00.000000,Long Buildups,Shorts Buildups


In [ ]:
destination_table = 'Titania_Dataset.' + str('')

In [22]:
import pandas as pd

data = [[18,'Male','Reddy'],[20,None,'Teja'],[22,'Male','Sai'],[20,'Male','Sai Teja Reddy']]

data = pd.DataFrame(data,columns = ['Age','Gender','Name'])

In [27]:
data

,Age,Gender,Name,Gendermap
0,18,Male,Reddy,1.0
1,20,None,Teja,NaN
2,22,Male,Sai,1.0
3,20,Male,Sai Teja Reddy,1.0


In [24]:
data['Gendermap'] = data.Gender.map({'Male':1,'Female':0})

In [25]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors = 2)

df_filled = imputer.fit_transform(data[['Age','Gendermap']])

In [26]:
df_filled

array([[18.,  1.],
       [20.,  1.],
       [22.,  1.],
       [20.,  1.]])